In [1]:
%load_ext autoreload
%autoreload 2

https://openaccess.uoc.edu/bitstream/10609/148363/3/miravecaTFG0623memoria.pdf

https://www.portalinmobiliario.com/robots.txt


SAAS
https://github.com/Saas-Starter-Kit/Saas-Kit-prisma?tab=readme-ov-file


In [2]:
import numpy as np
import pandas as pd
from webscrapper_portal_inmobiliario import WebScraperPortalInmobiliario

### TODO:
- ~~trackear propiedades en el tiempo para el mismo sector  SE HACE CON LA [LAT LONG]~~
- ~~Sortear baneos de ip por muchas request - sectores con muchas propiedades~~
https://brightdata.com/cp/zones/residential_proxy1/edit 8 dolares el GB --> SOLUCION
- ~~crear tabla intetmedia con 3 tablas y generar id unico...~~
- ~~buscar estandar para nombres de columnas en tablas, en ingles... sin mayusculas~~
- ~~Hacer que se descarguen las cards urls totales [venta + arriendo]~~
- ~~Los polygonos se agregan a una tabla en la base de datos... con una funcion~~
- ~~para correr el programa se ejecuta otra funcion que leera los polygoinos de la base de datos~~
- Que funcione descargando para multiples polygonos
- Modo descarga casas y departamentos en conjunto
- ejecutar todos los links en tread
---
- Calculos de estadicticas de rentabilidad
- rankear propiedades basado en esto
- grafico mapa donde se encuentran estas propiedades
- extraer imagenes del TOP 3 Propiedades
- plotear en single plot matplotlib. una especie de reporte automatico.

### fixes
- dias desde publicacion fallando y retornando cero


In [3]:
WSPI = WebScraperPortalInmobiliario()

## MAP HANDLING

CREATE NEW MAP GROUP

make a create, delete and edit cluster map function, diferent one for each action

In [20]:
WSPI.insert_new_cluser_map("Diego_portales",None)

LIST MAPS AVALIABLE

In [21]:
WSPI.list_db_maps()

  mapID       geo_ref_name
0     1  general_arriagada
1     2     Diego_portales


LOAD A SELECTED MAP INTO VISUALIZATION

In [23]:
WSPI.vis_map(selected_mapID=2)

Diego_portales --> Sucess!


Map(center=[-33.557610499999996, -70.55605405555555], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [24]:
WSPI.start_download(selected_mapID=2)

  0%|          | 0/100 [00:00<?, ?it/s]

proxy enabled!, random session id:  0.1617099416967237


# DATA MANAGE
------------------------------------------

In [25]:
WSPI.get_df_caprates(map_id=2)

analizando casa
con 2.0 dormitorios, se puede arrendar en aproximadamente 365000, analizado desde 2 propiedades
con 3.0 dormitorios, se puede arrendar en aproximadamente 685000, analizado desde 4 propiedades
con 4.0 dormitorios, se puede arrendar en aproximadamente 950000, analizado desde 3 propiedades
con 6.0 dormitorios, se puede arrendar en aproximadamente 200000, analizado desde 1 propiedades
precio/m2 de la zona es 6275 CLP , 0.17 UF
-------------------------------------
analizando departamento
con 1.0 dormitorios, se puede arrendar en aproximadamente 330000, analizado desde 7 propiedades
con 2.0 dormitorios, se puede arrendar en aproximadamente 430000, analizado desde 15 propiedades
con 3.0 dormitorios, se puede arrendar en aproximadamente 492000, analizado desde 3 propiedades
precio/m2 de la zona es 8326 CLP , 0.22 UF
-------------------------------------


In [26]:
WSPI.analysis_results["df_caprate_casa"]

,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,cantidad_pisos_edificio,piso_unidad,tipo_inmueble,orientacion,gastos_comunes,link,listed,Price,Price_UF,tipo_operacion,Date,caprate_bruto
11,28.0,4.0,3.0,145.0,115.0,2.0,2.0,0.0,2.0,NaN,casa,P,5000.0,https://www.portalinmobiliario.com/MLC-2665646...,1,1.552311e+08,4100.00,venta,2024-09-17 15:00:18.217868,0.073439
12,150.0,3.0,2.0,135.0,87.0,4.0,1.0,22.0,2.0,NaN,casa,-,0.0,https://www.portalinmobiliario.com/MLC-2383559...,1,1.500000e+08,3961.84,venta,2024-09-17 15:00:18.232565,0.054800
13,24.0,4.0,2.0,200.0,129.0,1.0,0.0,15.0,2.0,NaN,casa,SO,NaN,https://www.portalinmobiliario.com/MLC-1517915...,1,1.900000e+08,5018.32,venta,2024-09-17 15:00:18.243693,0.060000
14,42.0,4.0,2.0,237.0,88.0,2.0,1.0,0.0,2.0,NaN,casa,Sur-oriente-poniente,0.0,https://www.portalinmobiliario.com/MLC-1511704...,1,2.082368e+08,5500.00,venta,2024-09-17 15:00:18.257732,0.054745
15,270.0,3.0,2.0,209.0,77.0,2.0,0.0,26.0,1.0,NaN,casa,Nor-poniente,0.0,https://www.portalinmobiliario.com/MLC-1444957...,1,1.650000e+08,4358.02,venta,2024-09-17 15:00:18.269204,0.049818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,20.0,4.0,3.0,272.0,167.2,2.0,0.0,0.0,2.0,NaN,casa,NOSP,NaN,https://www.portalinmobiliario.com/MLC-2676084...,1,2.540000e+08,6708.71,venta,2024-09-17 15:00:19.192581,0.044882
95,14.0,5.0,3.0,167.0,150.0,2.0,1.0,35.0,2.0,NaN,casa,NOSP,0.0,https://www.portalinmobiliario.com/MLC-2685351...,1,1.980000e+08,5229.62,venta,2024-09-17 15:00:19.203373,NaN
96,60.0,3.0,2.0,246.0,110.0,3.0,1.0,22.0,2.0,NaN,casa,P,0.0,https://www.portalinmobiliario.com/MLC-2698975...,1,2.002860e+08,5290.00,venta,2024-09-17 15:00:19.213746,0.041041
97,5.0,3.0,1.0,110.0,80.0,2.0,NaN,NaN,2.0,NaN,casa,None,NaN,https://www.portalinmobiliario.com/MLC-2694708...,1,1.350510e+08,3567.00,venta,2024-09-17 15:00:19.223819,0.060866


In [19]:
WSPI.analysis_results["df_caprate_departamento"]

,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,cantidad_pisos_edificio,piso_unidad,tipo_inmueble,orientacion,gastos_comunes,link,listed,Price,Price_UF,tipo_operacion,Date,caprate_bruto
41,15.0,1.0,1.0,46.99,43.43,0.0,0.0,1.0,12.0,11.0,departamento,Poniente,58000.0,https://www.portalinmobiliario.com/MLC-2683367...,1,1.013924e+08,2678.00,venta,2024-09-17 14:34:38.045777,0.038464
42,14.0,2.0,2.0,77.25,72.71,0.0,0.0,1.0,11.0,0.0,departamento,Oriente,90000.0,https://www.portalinmobiliario.com/MLC-1521206...,1,1.562155e+08,4126.00,venta,2024-09-17 14:34:38.055778,0.033031
43,330.0,2.0,1.0,52.00,48.00,1.0,0.0,10.0,4.0,4.0,departamento,P,50000.0,https://www.portalinmobiliario.com/MLC-2370716...,1,1.173698e+08,3100.00,venta,2024-09-17 14:34:38.068777,0.043964
44,60.0,2.0,1.0,62.00,62.00,1.0,1.0,25.0,7.0,0.0,departamento,-,90000.0,https://www.portalinmobiliario.com/MLC-1503001...,1,1.270000e+08,3354.35,venta,2024-09-17 14:34:38.079779,0.040630
45,14.0,2.0,2.0,54.00,50.00,1.0,1.0,2.0,0.0,2.0,departamento,Nor-Oriente,70000.0,https://www.portalinmobiliario.com/MLC-2683985...,1,1.192629e+08,3150.00,venta,2024-09-17 14:34:38.089780,0.043266
46,150.0,2.0,2.0,58.00,54.00,1.0,1.0,2.0,15.0,0.0,departamento,N,0.0,https://www.portalinmobiliario.com/MLC-1474290...,1,1.094190e+08,2890.00,venta,2024-09-17 14:34:38.101777,0.047158
47,11.0,2.0,2.0,77.25,72.71,1.0,1.0,2.0,11.0,5.0,departamento,O,70000.0,https://www.portalinmobiliario.com/MLC-2688186...,1,1.699970e+08,4490.00,venta,2024-09-17 14:34:38.112780,0.030353
48,120.0,3.0,1.0,55.00,55.00,1.0,0.0,20.0,0.0,2.0,departamento,O,26000.0,https://www.portalinmobiliario.com/MLC-1478971...,1,7.534387e+07,1990.00,venta,2024-09-17 14:34:38.124779,0.063708
49,0.0,3.0,1.0,56.00,56.00,0.0,0.0,17.0,17.0,3023.0,departamento,SP,NaN,https://www.portalinmobiliario.com/MLC-1473135...,1,1.070000e+08,2826.11,venta,2024-09-17 14:34:38.133778,0.044860


# APRENDIZAJES


webdriver
2. es bueno reiniciar y cerrar el driver
2. el estado de la pagina (restricciones) cambiara solo con el refresh!!
3. Si se aplica un init y luego get/request no retornara NADA (PUEDE QUE SI PERO EN GENERAL NO PORQUE ES UN USER NUEVO SE ASOCIARA CON WEBSCRAPPING)
4. el orden correcto sera

close--> init--> request (primera vez ,  alta probabilida de falla) -->  refresh (recarga y se puede obtener la tabla y ubicaciion)